In [8]:
import json
import requests
import pandas as pd
import os
import datetime
import psycopg2
from pymongo import MongoClient
from io import StringIO

In [9]:
#Separate each section using Markdown etc.

In [10]:
#setting a directory temporarily
os.chdir(r"C:\Users\pat\Desktop\Databases and Analytics Programming")

In [11]:
#initializing mongo
client = MongoClient()
client = MongoClient("192.168.56.30", 27017)

#targeting desired weather db on Mongo
weather_db = client["DAP_project"]


In [12]:
#function that converts dates into seconds since 1970-01-01
#it's easier to use this format with the API
def convert_datetime(datetime_obj):
    dif = (datetime_obj - datetime.datetime(1970,1,1))
    secs_since_1970 = int(dif.total_seconds())
    return secs_since_1970


def revert_datetime(time):
    reverted = datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d %H:%M:%S")
    return reverted

In [13]:
#for historic download, i want these dates in 'seconds since 1970' format
start1 = convert_datetime(datetime.datetime(2011,1,1))
end1 = convert_datetime(datetime.datetime(2011,5,31))

#I also want the same data for 2020. Future refreshes will be automatic
start2 = convert_datetime(datetime.datetime(2020,1,1))
end2 = convert_datetime(datetime.datetime(2020,4, 3))


In [14]:
#weather api call. Required a separate call for each day if want hstory
#Therefore, will create a list of dates I want, and call this for each
#project is only looking at Dublin, no need to generalize it
def get_weather(time):
    
    dublin_loc = "53.3498,-6.2603"
    #I haven't saved the api key here, as I'd rather it not be called
    #Will put it back in when it's time to submit so Michael can use it.
    #api_key = ""    
    #building the url
    url = "https://dark-sky.p.rapidapi.com/"+dublin_loc+","+time
    
    #standard parameters for the darksky api (source = documentation)
    parameters = {"lang":"en","units":"auto"}
    
    headers = {
            'x-rapidapi-host': "dark-sky.p.rapidapi.com",
            'x-rapidapi-key': api_key
            }
    
    #the entire call stuck together
    response = requests.request("GET", url, headers=headers, params=parameters)
    
    return json.loads(response.text)

In [15]:
#generating the list of times which I fill feed through the api call
def generate_times(start, end):
    
    #useful for calculating days + generating the times
    sec_in_day = 86400
    dif_seconds = end - start
    
    counter = start
    time_list = []
    time_list.append(counter)
    while counter < end:
        time_list.append(counter + sec_in_day)
        counter += sec_in_day
    
    return time_list


In [16]:
#generates a list of dicts to post to mongo in one go
def data_retrieval(time_list):
    
    dump = []
    
    for time in time_list:
        try:
            dump.append(get_weather(str(time)))
        except:
            dump.append("error retrieving data for", str(time))
            continue
    
    return dump

In [17]:
#posts the list generated by func above 
def post_data(post_list):
    
    #Named posts before I knew could be called anything. no point changing
    posts = weather_db.posts
    posts.insert_many(post_list) 
    
    return 


In [18]:
#dumps the data retrieved from the api call into Mongo.
def data_dump(start, end):
    
    #takes in the times between the dates of interest
    time_list = generate_times(start, end)
    
    #gets the data from api
    retrieved_data = data_retrieval(time_list)
    
    #posts it
    post_data(retrieved_data)
    
    #only return it for testing purposes, and if I need to save the data
    #as backup
    return retrieved_data

In [19]:
#The code commented out below is the initial commands I used to call the api
#and transfer the data into Mongo. Calling the api costs money based on # of
#calls, so please don't call unless you absolutely have to.
#For Michael: the refresh_parse_push function does the same thing but for
#new data, it also has the postgres part in it. Please use this to verify it all
#works
    
'''
json_list_2011 = data_dump(start1, end1)
json_list_2020 = data_dump(start2, end2)

#quick check to see if all the data was loaded correctly
item_count = 0
for item in weather_db.posts.find():
    item_count +=1

#I expect 151 + 94 days worth of data
item_count == 245
'''

'\njson_list_2011 = data_dump(start1, end1)\njson_list_2020 = data_dump(start2, end2)\n\n#quick check to see if all the data was loaded correctly\nitem_count = 0\nfor item in weather_db.posts.find():\n    item_count +=1\n\n#I expect 151 + 94 days worth of data\nitem_count == 245\n'

In [20]:
#Code for reading data from mongo, and parsing it into a dataframe object
#I use this function to export to csv, and to prepare data for loading into
#postgres
def pull_parse(entity, start = 0):
    
    #initializing a dataframe
    df = pd.DataFrame()
    
    #I only want new data from mongo, unless this is the first time pulling
    #from the db. The function will know, as will be seen from functions below
    records = entity.find({"hourly.data.time": {"$gte": start}})
    
    for record in records:
        #each day has data stored in hourly sub components
        for hour in record["hourly"]["data"]:
            df = df.append(hour, ignore_index = True)
        
    #removed this as it is too sparsely populated to be useful. 4995/5000 NA
    try:
        df = df.drop(["precipAccumulation"], axis = 1)
    except:
        print("the column isn't there")
    
    finally:
        #coding missing values in ozone as 999. THis is because ozone is only
        #missing from older records.
        df.ozone = df.ozone.fillna(999)
        #filling missing vals for precip type, where intensity = 0
        df[df.iloc[:, 5] == 0] = df[df.iloc[:, 5] == 0].fillna("no_precip")
        #best approximation for missing values is the previous or the next
        #I chose backfill because the newer data tends to be more complete
        df = df.fillna(method = "bfill")

        #allows for storage of these vars as smallint, saves space.
        df.uvIndex = df.uvIndex.astype("int")
        df.windBearing = df.windBearing.astype("int")
        
        #reverting time to a better format for the db
        df["time"] = df["time"].apply(lambda x: revert_datetime(x))
        
        #writes to csv, with the max time in the filename. so I don't overwrite.
        #If something goes wrong, I have the file stored as backup/to inspect
        file = "weather_dataframe - "+str(datetime.date.today())+".csv"
        df.to_csv(file, index = False)
        
    return df


In [21]:
#This is for refreshing the data after the initial dump. It works on it's own
#to push data to mongo. But it is also called later in the refresh_parse_push
#function. I only want to pull new data generated since the last refresh
def data_refresh():
    #targetting the relevant db. not a func arg, as I always want this as targ
    entity = weather_db.posts
    #finds the last datetime already in the db
    last_refresh = entity.find_one(sort = [("hourly.data.time", -1)])
    last_time = last_refresh["hourly"]["data"][-1]["time"]
    
    #sets the start date to be the next hour. (if db already has 10pm, new
    #start date is 11pm)
    start = int(last_time+3600)
    
    #set end to yesterday 00:00:00. api calls the next 24 hours
    yesterday = datetime.date.today() - datetime.timedelta(days = 1)
    end = yesterday.strftime("%Y-%m-%d %H:%M:%S")
    end = datetime.datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    end = convert_datetime(end)
    
    #decided to retrieve the json list just incase I need it
    retrieved_data = data_dump(start, end)

    refresh_df = pull_parse(entity, start)
    
    #decided to return both of these, in case need to check functionality
    return refresh_df, retrieved_data



In [22]:
# Postgres Database Creation and Population
    
#makes connecting to db a bit quicker/cleaner
def db_connection(databaseName):
    
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = str(databaseName))
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    

    return dbConnection, dbCursor

In [23]:
#enter either "Select" or "Define". Not going to need to manipulate.
def enter_query(databaseName, queryString, queryType = "Select"):
    
    try:
        dbConnection, dbCursor = db_connection(databaseName)
        if queryType == "Select":
            #reads the query as a dataframe, much easier to work with
            results = pd.io.sql.read_sql_query(queryString, dbConnection)
        else:
            #otherwise just executes the datadefinition query
            dbCursor.execute(queryString)
            results = "query was in definition mode"
        
    except (Exception, psycopg2.Error) as dbError:
        print("Error:", dbError)
       
    finally:
        if(dbConnection): dbConnection.close()

    return results

In [ ]:
#Creating the project database
#createDatabaseString = 'CREATE DATABASE dap_project;'
#enter_query("postgres", queryString = createDatabaseString, queryType = "Define")

In [50]:
#datatypes:
#smallint for the ints, (the max values for these are 7 and 359, in this data)
#numeric for fixed decimal
#decided to double the current max length for varchars, just to be safe
#but it is possible, so not worth risking it for 1/2 extra bytes
#Set the Default for Location to be Dublin, because I forgot to store it
#in earlier functs + no point changing really.

#dropping and creating the db was quicker/easier than altering
dropString = "DROP TABLE weather;"
enter_query("dap_project", dropString, "Define")
    
weatherCreateString = """
CREATE TABLE weather(
        time timestamp,
        location varchar(30) DEFAULT 'Dublin',
        apparentTemperature numeric(4, 2),
        cloudCover numeric(3, 2) CHECK (cloudCover >= 0),
        dewPoint numeric(4, 2),
        humidity numeric(3, 2) CHECK (humidity >= 0),
        icon varchar(40),
        precipIntensity numeric(6, 4),
        precipProbability numeric(3, 2) CHECK (precipProbability >= 0),
        pressure numeric(5, 1) CHECK (pressure > 0),
        summary varchar(70),
        temperature numeric(4, 2),
        uvIndex smallint CHECK (uvIndex >= 0),
        visibility numeric(5, 3) CHECK (visibility >= 0),
        windBearing smallint CHECK (windBearing >= 0),
        windGust numeric(4, 2) CHECK (windGust >= 0),
        windSpeed numeric(4, 2) CHECK (windSpeed >= 0),
        precipType varchar(30),
        ozone numeric(4, 1) CHECK (ozone > 0),
        PRIMARY KEY (time, location)
        );
"""
#Actual creation of db
enter_query("dap_project", weatherCreateString, "Define")

'query was in definition mode'

In [45]:
#I read in the csv for the initial data dump here, as I did not do this all
#in one day etc.
weather_df = pd.read_csv("weather dataframe.csv")

#doing the operations that I have now built into the parse function. I improved
#the parse function after I had already pulled and parsed the data, no real 
#nead to do it again.
weather_df.ozone = weather_df.ozone.fillna(999)
weather_df.loc[weather_df.precipIntensity == 0, "precipType"] = "no_precip"
weather_df = weather_df.fillna(method = "bfill")
weather_df.windBearing = weather_df.windBearing.astype("int")
weather_df.uvIndex = weather_df.uvIndex.astype("int")

In [17]:
#function for dumped from a dataframe into a target table.
def postgres_dump(target_database, target_table, dataframe):

    pd_df = dataframe
    
    dbConnection, dbCursor = db_connection(target_database)
    
    #copying is easier than entering the values one by one, otherwise I would
    #have to write a huge string, as my data has 19 columns
    sio = StringIO()
    sio.write(pd_df.to_csv(index = None, header = None))
    sio.seek(0)
    
    dbCursor.copy_from(sio, target_table, columns = pd_df.columns, sep = ",")
    dbConnection.commit()

    return print("process complete")


In [ ]:
#Initial loading of data.
#takes targets the dap_project db, targets the weather table, imports the csv file.
postgres_dump("dap_project", "weather", weather_df)

refreshed_df = pd.read_csv("weather_dataframe - 2020-04-06.csv")

postgres_dump("dap_project", "weather", refreshed_df)

In [223]:
#this function automates the refreshing of data. It will pull data from the
#api automatically, and then it will parse it to a csv format, and ultimately
#it will append it to the weather_table in Postgres

def refresh_parse_push(target_table):
    
    refreshed_df, refreshed_json = data_refresh()
    
    postgres_dump(target_table, refreshed_df)
    
    return refreshed_df, refreshed_json
    

In [ ]:
#testing refresh function

api_key = str(input("Enter the api key here (if not defined in func)"))

#refresh_test1_df, refresh_test1_json = refresh_parse_push("weather")


In [ ]:
#Jude Code

In [18]:
url = 'https://data.smartdublin.ie/dataset/10130831-d2a5-4f4f-b56b-03d8e2cc56c8/resource/9f8a9b41-60a9-4cbd-8630-865e51a18724/download/dcccivicofficeheatingdegreedaysp20130221-1853.csv'
try : 
    response = requests.get(url)
    csv_text = response.text
except Exception as err: 
    print("Error occured", err)

# name the file to be written
csv_file = 'heating.csv'

#write file to disk
with open(csv_file, 'w',newline='') as file: 
    file.write(csv_text)

In [19]:
# read downloadded file into pandas daraframe
import pandas as pd
df = pd.read_csv(url, skiprows = 1)

In [20]:
#inspect the dataframe
df.head()



,Date,Values,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,07/11/2009,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.096250,0.095937,0.098854,0.101354,0.103646,0.105104,0.106146,0.106979,0.107604,0.107917
1,08/11/2009,96,0.106875,0.107708,0.107708,0.106771,0.105729,0.105104,0.106042,0.103854,...,0.106771,0.106458,0.106771,0.106667,0.108854,0.110417,0.111562,0.110417,0.112708,0.115938
2,09/11/2009,96,0.118958,0.117708,0.116563,0.117292,0.116979,0.118229,0.117500,0.116979,...,0.065313,0.065208,0.065208,0.065313,0.065313,0.065833,0.065729,0.065313,0.064792,0.064271
3,10/11/2009,96,0.063750,0.063542,0.062708,0.062812,0.063021,0.063021,0.061979,0.060833,...,0.104688,0.105729,0.106979,0.108542,0.110000,0.111979,0.112917,0.112292,0.114062,0.116563
4,11/11/2009,96,0.118021,0.118333,0.120208,0.119479,0.117708,0.117396,0.116667,0.114479,...,0.052708,0.052292,0.052396,0.053229,0.056458,0.059479,0.065313,0.067396,0.068437,0.068542


In [21]:
# get dimensions of raw dataset 
df.shape




(1202, 98)

In [22]:
# find out how many values are missing
df.isna().sum().sum()
# total of 119 fifteen mins readings missing out of more than 115K observations



119

In [23]:

# find out the rows with missing values
df[df.isna().any(axis=1)]
# 10 rows have some missing values




,Date,Values,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,07/11/2009,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.096250,0.095937,0.098854,0.101354,0.103646,0.105104,0.106146,0.106979,0.107604,0.107917
23,30/11/2009,96,0.125521,0.129271,0.130208,0.131042,0.132188,0.132500,0.132917,0.134167,...,0.148542,0.148750,0.145417,0.143438,0.143542,0.143542,0.144167,0.144167,0.148854,0.153750
24,01/12/2009,96,0.152083,0.151458,NaN,NaN,NaN,NaN,NaN,NaN,...,0.071562,0.075938,0.081562,0.084792,0.087187,0.087813,0.088229,0.087187,0.089375,0.091354
32,09/12/2009,96,0.053646,0.055313,0.055833,0.055625,0.056458,0.057083,0.058542,0.058437,...,0.088021,0.088229,0.088125,0.089271,0.089271,0.090104,0.091250,0.092396,0.093437,0.094479
68,14/01/2010,96,0.125833,0.128750,0.130104,0.129271,0.129896,0.134896,0.139375,0.144167,...,0.111458,0.108958,0.109167,0.108021,0.105000,0.102812,0.098333,0.095417,0.093437,0.092708
683,21/09/2011,96,0.045041,0.047922,0.048698,0.049489,0.050359,0.050993,0.051131,0.051318,...,0.043357,0.043813,0.044346,0.044423,0.043834,0.045131,0.047165,0.048516,0.050299,0.050752
722,30/10/2011,96,0.040349,0.039386,0.037034,NaN,NaN,NaN,NaN,0.034762,...,0.010910,0.010751,0.011269,0.014303,0.015343,0.014905,0.014571,0.013212,0.013205,0.015514
793,09/01/2012,96,0.055513,0.055991,0.055236,0.054862,0.055591,0.056324,0.056096,0.057994,...,0.080478,0.079767,0.079091,0.078261,0.078062,0.078088,0.077993,0.078751,0.078369,0.079149
891,16/04/2012,96,0.101918,0.101953,0.103146,0.104129,0.104749,0.105173,0.103362,0.102619,...,0.065768,0.064369,0.065288,0.065093,0.066454,0.065696,0.063404,0.062938,0.061739,0.060999
1086,28/10/2012,96,0.094084,0.092049,0.089251,NaN,NaN,NaN,NaN,0.087206,...,0.074546,0.074752,0.074784,0.074123,0.075947,0.078298,0.079306,0.079618,0.080231,0.080620


In [24]:
# use column means to fill the missing values
df.fillna(df.mean(), inplace=True)
df.head()



,Date,Values,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,07/11/2009,96,0.068573,0.069090,0.069483,0.069916,0.070368,0.070830,0.071209,0.071484,...,0.096250,0.095937,0.098854,0.101354,0.103646,0.105104,0.106146,0.106979,0.107604,0.107917
1,08/11/2009,96,0.106875,0.107708,0.107708,0.106771,0.105729,0.105104,0.106042,0.103854,...,0.106771,0.106458,0.106771,0.106667,0.108854,0.110417,0.111562,0.110417,0.112708,0.115938
2,09/11/2009,96,0.118958,0.117708,0.116563,0.117292,0.116979,0.118229,0.117500,0.116979,...,0.065313,0.065208,0.065208,0.065313,0.065313,0.065833,0.065729,0.065313,0.064792,0.064271
3,10/11/2009,96,0.063750,0.063542,0.062708,0.062812,0.063021,0.063021,0.061979,0.060833,...,0.104688,0.105729,0.106979,0.108542,0.110000,0.111979,0.112917,0.112292,0.114062,0.116563
4,11/11/2009,96,0.118021,0.118333,0.120208,0.119479,0.117708,0.117396,0.116667,0.114479,...,0.052708,0.052292,0.052396,0.053229,0.056458,0.059479,0.065313,0.067396,0.068437,0.068542


In [25]:
# get dimensions of clean data
df.shape



(1202, 98)

In [26]:
# get a list of all columns in data frame
col_names = list(df.columns)



In [27]:
# Heating degree day for the a particular day is the sum of all values from 00:00 to 23:45
# remove columns 'date' and 'values' and sum the rest 
sum_columns = col_names[2:]



In [28]:
#create a new empty dataframe for daily data
df_daily = pd.DataFrame() 



In [29]:
# sum columns 00.00 to 23.45 and put the value in a new column hdd (stands for heating degree day)
df_daily['Date'] = df['Date']
df_daily['hdd'] = df[sum_columns].sum(axis=1)




In [30]:
#inspect dataframe
df_daily.head()

df_daily.shape



(1202, 2)

In [ ]:
#I don't enter this as it creates a separate database, I change the target to my own
'''# connect to postgres and create a database called 'project'
import psycopg2
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = ""192.168.56.30",
    port = "27017",
    database = "weather")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE project;')
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

'''


In [31]:
# create db schema for daily hdd readings, date as primary key
createString = """
CREATE TABLE dailyhdd(
date DATE PRIMARY KEY,
hdd numeric
);
"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()


In [32]:

# insert values from dataframe into database table
import pandas.io.sql as sqlio

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    for i,row in df_daily.iterrows():
        # on each row zeroth index has the date and 1st index has hdd value
        insertString = "INSERT INTO dailyhdd VALUES(to_date('{}','DD/MM/YYYY'), {})".format(row[0], row[1])
        dbCursor.execute(insertString)
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()
    


In [34]:
# query the dailyhdd table
sql = "SELECT * FROM dailyhdd"

try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    df_heating = sqlio.read_sql_query(sql, dbConnection)
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()





In [35]:
# inspect the result
df_heating


,date,hdd
0,2009-11-07,7.390832
1,2009-11-08,8.025417
2,2009-11-09,7.761250
3,2009-11-10,6.895729
4,2009-11-11,7.344792
...,...,...
1197,2013-02-16,6.998725
1198,2013-02-17,7.832932
1199,2013-02-18,9.209368
1200,2013-02-19,10.676997


In [36]:


# raw data dataframe df
df.head()



,Date,Values,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,07/11/2009,96,0.068573,0.069090,0.069483,0.069916,0.070368,0.070830,0.071209,0.071484,...,0.096250,0.095937,0.098854,0.101354,0.103646,0.105104,0.106146,0.106979,0.107604,0.107917
1,08/11/2009,96,0.106875,0.107708,0.107708,0.106771,0.105729,0.105104,0.106042,0.103854,...,0.106771,0.106458,0.106771,0.106667,0.108854,0.110417,0.111562,0.110417,0.112708,0.115938
2,09/11/2009,96,0.118958,0.117708,0.116563,0.117292,0.116979,0.118229,0.117500,0.116979,...,0.065313,0.065208,0.065208,0.065313,0.065313,0.065833,0.065729,0.065313,0.064792,0.064271
3,10/11/2009,96,0.063750,0.063542,0.062708,0.062812,0.063021,0.063021,0.061979,0.060833,...,0.104688,0.105729,0.106979,0.108542,0.110000,0.111979,0.112917,0.112292,0.114062,0.116563
4,11/11/2009,96,0.118021,0.118333,0.120208,0.119479,0.117708,0.117396,0.116667,0.114479,...,0.052708,0.052292,0.052396,0.053229,0.056458,0.059479,0.065313,0.067396,0.068437,0.068542


In [37]:
# create a new dataframe with only 15 mins interval hdd values by selecting only the columns with 15-mins readings
df2 = df.iloc[:,2:98] 
df2.head()



,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,0.068573,0.069090,0.069483,0.069916,0.070368,0.070830,0.071209,0.071484,0.071762,0.072100,...,0.096250,0.095937,0.098854,0.101354,0.103646,0.105104,0.106146,0.106979,0.107604,0.107917
1,0.106875,0.107708,0.107708,0.106771,0.105729,0.105104,0.106042,0.103854,0.100104,0.094271,...,0.106771,0.106458,0.106771,0.106667,0.108854,0.110417,0.111562,0.110417,0.112708,0.115938
2,0.118958,0.117708,0.116563,0.117292,0.116979,0.118229,0.117500,0.116979,0.119063,0.121250,...,0.065313,0.065208,0.065208,0.065313,0.065313,0.065833,0.065729,0.065313,0.064792,0.064271
3,0.063750,0.063542,0.062708,0.062812,0.063021,0.063021,0.061979,0.060833,0.061250,0.063542,...,0.104688,0.105729,0.106979,0.108542,0.110000,0.111979,0.112917,0.112292,0.114062,0.116563
4,0.118021,0.118333,0.120208,0.119479,0.117708,0.117396,0.116667,0.114479,0.109896,0.108229,...,0.052708,0.052292,0.052396,0.053229,0.056458,0.059479,0.065313,0.067396,0.068437,0.068542


In [38]:
# calculate hourly readings by summing up of every 4 rows. Save this to a new dataframe. 
df_hourly = df2.groupby([[i//4 for i in range(0,96)]], axis = 1).sum()
df_hourly.head()



,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.277063,0.283891,0.289174,0.294929,0.298257,0.295683,0.284189,0.265660,0.316466,0.328958,...,0.261250,0.283750,0.295104,0.308854,0.337083,0.335938,0.360208,0.381979,0.408958,0.428646
1,0.429063,0.420729,0.386042,0.407708,0.386875,0.385104,0.415000,0.424583,0.414062,0.355000,...,0.103958,0.172188,0.264063,0.319063,0.320937,0.375729,0.411979,0.422292,0.432708,0.450625
2,0.470521,0.469687,0.488646,0.502500,0.517292,0.526042,0.495000,0.481042,0.417292,0.329167,...,0.169687,0.184167,0.207396,0.217708,0.230625,0.259063,0.267708,0.262708,0.261667,0.260104
3,0.252813,0.248854,0.256563,0.270000,0.280938,0.294792,0.316875,0.360000,0.381250,0.341042,...,0.113854,0.173229,0.229375,0.271146,0.319479,0.347812,0.393750,0.416042,0.437500,0.455833
4,0.476042,0.466250,0.430521,0.418958,0.451979,0.488958,0.484167,0.459167,0.387396,0.311979,...,0.186563,0.198229,0.220729,0.209271,0.199688,0.201250,0.202917,0.210521,0.221562,0.269688


In [39]:
# rename col_names and insert date columns at index 0
col_names = ['00:00:00', '01:00:00', '02:00:00', '03:00:00', '04:00:00','05:00:00','06:00:00','07:00:00','08:00:00','09:00:00','10:00:00','11:00:00','12:00:00',
            '13:00:00','14:00:00','15:00:00','16:00:00','17:00:00','18:00:00','19:00:00','20:00:00','21:00:00','22:00:00','23:00:00']
df_hourly.columns = col_names
df_hourly.insert(0, 'Date', df['Date'])

# inspect hourly hdd dataframe
df_hourly.head()



,Date,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
0,07/11/2009,0.277063,0.283891,0.289174,0.294929,0.298257,0.295683,0.284189,0.265660,0.316466,...,0.261250,0.283750,0.295104,0.308854,0.337083,0.335938,0.360208,0.381979,0.408958,0.428646
1,08/11/2009,0.429063,0.420729,0.386042,0.407708,0.386875,0.385104,0.415000,0.424583,0.414062,...,0.103958,0.172188,0.264063,0.319063,0.320937,0.375729,0.411979,0.422292,0.432708,0.450625
2,09/11/2009,0.470521,0.469687,0.488646,0.502500,0.517292,0.526042,0.495000,0.481042,0.417292,...,0.169687,0.184167,0.207396,0.217708,0.230625,0.259063,0.267708,0.262708,0.261667,0.260104
3,10/11/2009,0.252813,0.248854,0.256563,0.270000,0.280938,0.294792,0.316875,0.360000,0.381250,...,0.113854,0.173229,0.229375,0.271146,0.319479,0.347812,0.393750,0.416042,0.437500,0.455833
4,11/11/2009,0.476042,0.466250,0.430521,0.418958,0.451979,0.488958,0.484167,0.459167,0.387396,...,0.186563,0.198229,0.220729,0.209271,0.199688,0.201250,0.202917,0.210521,0.221562,0.269688


In [42]:
#create a new table in postgres to save hourly hdd data
createString = """
CREATE TABLE hourlyhdd(
date_time timestamp PRIMARY KEY,
hdd numeric
);
"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [43]:
# no of columns
col_names = df_hourly.columns # make a list of all the column names in the dataframe 
totalcolumns = len(col_names)
totalcolumns


25

In [44]:
# insert values from dataframe into hourlyhdd table
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    
    for i,row in df_hourly.iterrows(): 
        for j in range(1,totalcolumns) : # index 0 is 'date'. we want to add only the hourly data 
            date_time = row[0] + " " + col_names[j]  # eg : date_time = '07/11/2009 01:00:00'
            insertString = "INSERT INTO hourlyhdd VALUES('{}', {})".format(date_time, row[j])
            dbCursor.execute(insertString)
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [45]:

# query the hourlyhdd table
sql = "SELECT * FROM hourlyhdd"

try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    df_hourlyhdd = sqlio.read_sql_query(sql, dbConnection)
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()



In [47]:
#inspect data
df_hourlyhdd.shape







(28848, 2)

In [48]:

df_hourlyhdd.head()

,date_time,hdd
0,2009-11-07 00:00:00,0.277063
1,2009-11-07 01:00:00,0.283891
2,2009-11-07 02:00:00,0.289174
3,2009-11-07 03:00:00,0.294929
4,2009-11-07 04:00:00,0.298257


In [49]:
# Vasiliki Code - Pollution data retrieval

Timestamp('2013-02-20 23:00:00')

In [ ]:
################################## IMPORT THE DATA INTO PYTHON ##################################
# download the CO, SO2 and NO2 csv files and save them locally 
import requests 

url_CO = 'https://data.smartdublin.ie/dataset/c267242a-b0ee-4139-9cf7-c82995b8a2a3/resource/f3d9fddb-3337-4c99-90d5-c32fcfeb1e6a/download/dublin-city-council-co-2011p20110929-1048.csv'
url_SO2 = 'https://data.smartdublin.ie/dataset/c267242a-b0ee-4139-9cf7-c82995b8a2a3/resource/77cd9170-75bd-4e0f-a29d-513ecee9c607/download/dublin-city-council-so2-2011p20110929-1047.csv'
url_NO2 = 'https://data.smartdublin.ie/dataset/c267242a-b0ee-4139-9cf7-c82995b8a2a3/resource/fbbdf36b-f5b6-47ab-b7b1-f3a9b8bd388e/download/dublin-city-council-no-no2-2011p20110929-1045.csv'

try : 
    response1 = requests.get(url_CO)
    response2 = requests.get(url_SO2)
    response3 = requests.get(url_NO2)
    csv_text1 = response1.text
    csv_text2 = response2.text
    csv_text3 = response3.text
    
except Exception as err: 
    print("Error occured", err)

# name the files to be written
csv_file1 = 'CO_2011.csv'
csv_file2 = 'SO2_2011.csv'
csv_file3 = 'NO2_2011.csv'

#write files to disk
with open(csv_file1, 'w',newline='') as file: 
    file.write(csv_text1)
with open(csv_file2, 'w',newline='') as file: 
    file.write(csv_text2)
with open(csv_file3, 'w',newline='') as file: 
    file.write(csv_text3)

In [ ]:
# read downloaded files into pandas daraframe
import pandas as pd
import numpy as np
df_CO = pd.read_csv(url_CO,skiprows=5)
df_SO2 = pd.read_csv(url_SO2,skiprows=4)
df_NO2 = pd.read_csv(url_NO2,skiprows=5)

In [ ]:
# get dimensions of raw datasets
print(df_CO.shape)
print(df_SO2.shape)
print(df_NO2.shape)

In [ ]:
################################## Transforming CO DATASET ##################################

In [ ]:
#inspect the dataframe for CO
df_CO.head()

In [6]:
df_CO = df_CO[:3632] #keep top 3632

In [7]:
# Drop first 8 rows that refer to 2010
df_CO.drop(df_CO.index[:8], inplace=True)

In [8]:
df_CO.head()

,Site,Unnamed: 1,Wood Quay (Winetavern St.),Unnamed: 3,Unnamed: 4,Unnamed: 5,Coleraine Street,Unnamed: 7,Unnamed: 8,Unnamed: 9
8,01/01/2011,0,NaN,0.2,NaN,NaN,NaN,#DIV/0!,NaN,NaN
9,01/01/2011,1,0.2,0.1,2,NaN,0.5,0.5,2,NaN
10,01/01/2011,2,0.1,0.1,2,NaN,0.6,0.5,2,NaN
11,01/01/2011,3,0.1,0.1,2,NaN,0.6,0.6,2,NaN
12,01/01/2011,4,0.1,0.1,2,NaN,0.5,0.5,2,NaN


In [9]:
print(df_CO.columns)

Index(['Site', 'Unnamed: 1', '           Wood Quay (Winetavern St.)',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', '         Coleraine Street',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')


In [10]:
# Drop multiple columns based on column index and keep only CO of the 2 Streets
df_CO.drop(df_CO.columns[[3,4,5,7,8,9]], axis = 1, inplace=True)

In [11]:
# Rename the remaining columns
col_rename = {'Site': 'Date', 'Unnamed: 1': 'Time', 
                      '           Wood Quay (Winetavern St.)': 'WinetavernSt', '         Coleraine Street': 'ColeraineSt'} 
df_CO_renamed = df_CO.rename(columns=col_rename)
df_CO_renamed.head()

,Date,Time,WinetavernSt,ColeraineSt
8,01/01/2011,0,NaN,NaN
9,01/01/2011,1,0.2,0.5
10,01/01/2011,2,0.1,0.6
11,01/01/2011,3,0.1,0.6
12,01/01/2011,4,0.1,0.5


In [12]:
# Start index from 0 instead of 8
df_CO_renamed.reset_index(drop=True, inplace=True)

In [13]:
# get dimensions of transformed dataset
print(df_CO_renamed.shape)
print(df_CO_renamed.head())
df_CO_renamed.tail()

(3624, 4)
         Date Time WinetavernSt ColeraineSt
0  01/01/2011    0          NaN         NaN
1  01/01/2011    1          0.2         0.5
2  01/01/2011    2          0.1         0.6
3  01/01/2011    3          0.1         0.6
4  01/01/2011    4          0.1         0.5


,Date,Time,WinetavernSt,ColeraineSt
3619,31/05/2011,19,0,0.3
3620,31/05/2011,20,0,0.3
3621,31/05/2011,21,0,0.6
3622,31/05/2011,22,0,0.3
3623,31/05/2011,23,0,0.4


In [14]:
# Find the special characters/non numeric values in the dataframe CO
print(df_CO_renamed.iloc[:,2].unique())
print(df_CO_renamed.iloc[:,3].unique())

[nan '0.2' '0.1' '0' '0.3' 'CAL' '0.6' '0.7' '0.5' '0.4' '0.8' '0.9' '1.1'
 '1' '1.2' 'Zero Air' 'Cal' 'SERVICE' 'No data']
[nan '0.5' '0.6' '0.4' '0.3' '0.2' '0.7' '0.8' 'cal' '0.9' '1.2' '1.7'
 '1.9' '2.2' '1.1' '1.5' '0.1' '0' '1.3' '1' '2.1' 'no data' '1.8' '1.6'
 '1.4' '2.3' '2.5' '2.7' '2.6' '3' '2.9' '2' '3.4' '3.3' '2.4' '0.33'
 '-0.1' '-0.2']


In [15]:
# Find frequencies of missings and special characters
print(df_CO_renamed['WinetavernSt'].value_counts(dropna=False))
print(df_CO_renamed['ColeraineSt'].value_counts(dropna=False))

0           1495
0.1         1080
0.2          731
0.3          144
0.4           61
0.5           33
0.6           23
0.7           16
0.8            9
CAL            8
SERVICE        7
0.9            5
1              4
Zero Air       2
Cal            2
No data        1
1.1            1
1.2            1
NaN            1
Name: WinetavernSt, dtype: int64
0.4        727
0.3        590
0.5        514
0.2        430
0.6        349
0.7        240
0.1        153
0          152
0.8        109
-0.1        92
0.9         52
1           25
-0.2        20
1.3         17
1.1         17
1.2         16
1.4         13
1.7         13
1.6         13
cal         12
1.9         12
1.5         11
1.8          9
2.1          8
2.5          5
2            4
no data      3
2.7          3
2.2          2
3            2
2.3          2
2.6          2
3.4          2
2.9          1
0.33         1
2.4          1
3.3          1
NaN          1
Name: ColeraineSt, dtype: int64


In [16]:
# Find rows with special characters in WinetavernSt
print(df_CO_renamed[df_CO_renamed.WinetavernSt== 'CAL'])
print(df_CO_renamed[df_CO_renamed.WinetavernSt== 'Cal'])
print(df_CO_renamed[df_CO_renamed.WinetavernSt== 'No data'])
print(df_CO_renamed[df_CO_renamed.WinetavernSt== 'SERVICE'])
print(df_CO_renamed[df_CO_renamed.WinetavernSt== 'Zero Air'])

            Date Time WinetavernSt ColeraineSt
83    04/01/2011   11          CAL         0.4
84    04/01/2011   12          CAL         0.4
85    04/01/2011   13          CAL         0.4
780   02/02/2011   12          CAL         0.4
2271  05/04/2011   15          CAL         0.5
2272  05/04/2011   16          CAL         0.5
2942  03/05/2011   14          CAL         0.7
2943  03/05/2011   15          CAL         0.7
            Date Time WinetavernSt ColeraineSt
1668  11/03/2011   12          Cal         0.6
1669  11/03/2011   13          Cal         0.5
            Date Time WinetavernSt ColeraineSt
2042  27/03/2011    2      No data         0.3
            Date Time WinetavernSt ColeraineSt
1762  15/03/2011   10      SERVICE         0.4
1763  15/03/2011   11      SERVICE         0.4
1764  15/03/2011   12      SERVICE         0.4
1765  15/03/2011   13      SERVICE         0.4
1766  15/03/2011   14      SERVICE         0.5
1767  15/03/2011   15      SERVICE         0.5
1768  15/03/2

In [17]:
# Find rows with special characters in ColeraineSt
print(df_CO_renamed[df_CO_renamed.ColeraineSt== 'cal'])
print(df_CO_renamed[df_CO_renamed.ColeraineSt== 'no data'])

            Date Time WinetavernSt ColeraineSt
110   05/01/2011   14            0         cal
756   01/02/2011   12          0.1         cal
757   01/02/2011   13          0.1         cal
1621  09/03/2011   13          0.2         cal
1785  16/03/2011    9          0.4         cal
1786  16/03/2011   10          0.1         cal
1787  16/03/2011   11            0         cal
2292  06/04/2011   12          0.1         cal
2293  06/04/2011   13          0.1         cal
2962  04/05/2011   10            0         cal
3613  31/05/2011   13            0         cal
3614  31/05/2011   14            0         cal
            Date Time WinetavernSt ColeraineSt
427   18/01/2011   19          0.3     no data
428   18/01/2011   20          0.3     no data
2459  13/04/2011   11          0.1     no data


In [18]:
spec_char = ['No data','CAL','Cal','SERVICE','Zero Air','no data','cal']
for i in spec_char:
    df_CO_renamed = df_CO_renamed.applymap(lambda x: np.NaN if (type(x) is str and i in x) else x)
    print(df_CO_renamed[df_CO_renamed.WinetavernSt == i])
    print(df_CO_renamed[df_CO_renamed.ColeraineSt == i])

Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: 

In [19]:
# Find the total rows with missing values from the transformed dataframe
df_CO_renamed[df_CO_renamed.isna().any(axis=1)]
# 37 rows

,Date,Time,WinetavernSt,ColeraineSt
0,01/01/2011,0,NaN,NaN
83,04/01/2011,11,NaN,0.4
84,04/01/2011,12,NaN,0.4
85,04/01/2011,13,NaN,0.4
110,05/01/2011,14,0,NaN
427,18/01/2011,19,0.3,NaN
428,18/01/2011,20,0.3,NaN
756,01/02/2011,12,0.1,NaN
757,01/02/2011,13,0.1,NaN
780,02/02/2011,12,NaN,0.4


In [20]:
# find out how many values are missing
df_CO_renamed.isna().sum().sum()
# total of 37 missing out of 14,496 observations

37

In [21]:
# Rows that need to be transformed from NA to numeric
rows_excl_W = df_CO_renamed[df_CO_renamed[['WinetavernSt']].isna().any(axis=1)].index.tolist()
rows_excl_C = df_CO_renamed[df_CO_renamed[['ColeraineSt']].isna().any(axis=1)].index.tolist()
print(rows_excl_W) # number of values which will be excluded from mean calculation
print(rows_excl_C) # number of values which will be excluded from mean calculation

[0, 83, 84, 85, 780, 1644, 1645, 1668, 1669, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 2042, 2271, 2272, 2942, 2943]
[0, 110, 427, 428, 756, 757, 1621, 1785, 1786, 1787, 2292, 2293, 2459, 2962, 3613, 3614]


In [22]:
# Drop Date column for groupby Time calculation of mean
df_CO_trans = df_CO_renamed.drop(df_CO_renamed.columns[:1], axis = 1, inplace=False)

# Convert format of CO variables into numeric
df_CO_trans[["WinetavernSt"]] = df_CO_trans[["WinetavernSt"]].apply(pd.to_numeric)
df_CO_trans[["ColeraineSt"]] = df_CO_trans[["ColeraineSt"]].apply(pd.to_numeric)
df_CO_trans[["Time"]] = df_CO_trans[["Time"]].apply(pd.to_numeric)

# Assess if group means by Time have differences
print(df_CO_trans.groupby('Time').aggregate('mean'))

# Replace NAs/missing values with group means
df_CO_trans["WinetavernSt"] = df_CO_trans.groupby("Time")["WinetavernSt"].transform(lambda x: x.fillna(x.mean()))
df_CO_trans["ColeraineSt"] = df_CO_trans.groupby("Time")["ColeraineSt"].transform(lambda x: x.fillna(x.mean()))


      WinetavernSt  ColeraineSt
Time                           
0         0.120000     0.468200
1         0.113245     0.429139
2         0.107333     0.396026
3         0.092053     0.367550
4         0.088742     0.350331
5         0.084106     0.354967
6         0.078808     0.420530
7         0.084768     0.481457
8         0.112583     0.493377
9         0.133113     0.440667
10        0.115333     0.376510
11        0.102013     0.357718
12        0.086301     0.375168
13        0.082313     0.403401
14        0.081208     0.414094
15        0.083784     0.425828
16        0.091275     0.458278
17        0.117219     0.490066
18        0.133113     0.521854
19        0.129139     0.527333
20        0.117881     0.552667
21        0.127815     0.543046
22        0.125166     0.527152
23        0.130464     0.512583


In [23]:
# find out how many values of the transformed dataset are missing
df_CO_trans.isna().sum().sum()
# 0 missing values

0

In [24]:
df_CO_trans.head()
print(df_CO_trans.shape)

(3624, 3)


In [25]:
# Create a new variable called date_time which combines Date and Time from 2011/01/01 00:00:00 to 2011/05/31 23:00:00 
df_CO_trans.loc[:,'date_time'] = pd.date_range('2011/01/01', periods = 3624, freq ='H')
# Restore Date column for checking purposes
df_CO_trans.loc[:,'Date'] = df_CO_renamed.loc[:,'Date']


In [26]:
# check if it's working
df_CO_trans.iloc[740:758,:]

,Time,WinetavernSt,ColeraineSt,date_time,Date
740,20,0.0,0.300000,2011-01-31 20:00:00,31/01/2011
741,21,0.0,0.300000,2011-01-31 21:00:00,31/01/2011
742,22,0.0,0.200000,2011-01-31 22:00:00,31/01/2011
743,23,0.0,0.200000,2011-01-31 23:00:00,31/01/2011
744,0,0.0,0.200000,2011-02-01 00:00:00,01/02/2011
745,1,0.0,0.200000,2011-02-01 01:00:00,01/02/2011
746,2,0.0,0.300000,2011-02-01 02:00:00,01/02/2011
747,3,0.0,0.300000,2011-02-01 03:00:00,01/02/2011
748,4,0.0,0.300000,2011-02-01 04:00:00,01/02/2011
749,5,0.0,0.300000,2011-02-01 05:00:00,01/02/2011


In [27]:
# Drop Date and Time variables
df_CO_trans.drop(df_CO_trans.columns[[0,4]], axis = 1, inplace=True)

In [28]:
# Display head and tail of the current dataset
def display_n(df,n): 
    with pd.option_context('display.max_rows',n*2):
        display(df)
        
display_n(df_CO_trans,10)

,WinetavernSt,ColeraineSt,date_time
0,0.12,0.4682,2011-01-01 00:00:00
1,0.20,0.5000,2011-01-01 01:00:00
2,0.10,0.6000,2011-01-01 02:00:00
3,0.10,0.6000,2011-01-01 03:00:00
4,0.10,0.5000,2011-01-01 04:00:00
...,...,...,...
3619,0.00,0.3000,2011-05-31 19:00:00
3620,0.00,0.3000,2011-05-31 20:00:00
3621,0.00,0.6000,2011-05-31 21:00:00
3622,0.00,0.3000,2011-05-31 22:00:00


In [29]:
# Intermediate dataframes for concatenating them
df_CO_Wine = pd.concat([df_CO_trans['date_time'], df_CO_trans['WinetavernSt']], axis=1, keys=['date_time','WinetavernSt'])
df_CO_Cole = pd.concat([df_CO_trans['date_time'], df_CO_trans['ColeraineSt']], axis=1, keys=['date_time','ColeraineSt'])

# Rename columns for values as CO
df_CO_Wine = df_CO_Wine.rename(columns={"WinetavernSt": "CO"})
df_CO_Cole = df_CO_Cole.rename(columns={"ColeraineSt": "CO"})

# Create a variable as an ID for street
df_CO_Wine.loc[:,'Street'] = 'Winetavern'
df_CO_Cole.loc[:,'Street'] = 'Coleraine'

print(df_CO_Wine.head())
print(df_CO_Cole.head())

            date_time    CO      Street
0 2011-01-01 00:00:00  0.12  Winetavern
1 2011-01-01 01:00:00  0.20  Winetavern
2 2011-01-01 02:00:00  0.10  Winetavern
3 2011-01-01 03:00:00  0.10  Winetavern
4 2011-01-01 04:00:00  0.10  Winetavern
            date_time      CO     Street
0 2011-01-01 00:00:00  0.4682  Coleraine
1 2011-01-01 01:00:00  0.5000  Coleraine
2 2011-01-01 02:00:00  0.6000  Coleraine
3 2011-01-01 03:00:00  0.6000  Coleraine
4 2011-01-01 04:00:00  0.5000  Coleraine


In [30]:
# Create a final concatenated dataset with an ID variable called "Street"
df_CO_final = pd.concat([df_CO_Wine, df_CO_Cole], axis=0)
df_CO_final = df_CO_final[['date_time', 'Street','CO']]

# Reset index in order to have 0:7247
df_CO_final.reset_index(drop=True, inplace=True)
display_n(df_CO_final,10)

,date_time,Street,CO
0,2011-01-01 00:00:00,Winetavern,0.12
1,2011-01-01 01:00:00,Winetavern,0.20
2,2011-01-01 02:00:00,Winetavern,0.10
3,2011-01-01 03:00:00,Winetavern,0.10
4,2011-01-01 04:00:00,Winetavern,0.10
...,...,...,...
7243,2011-05-31 19:00:00,Coleraine,0.30
7244,2011-05-31 20:00:00,Coleraine,0.30
7245,2011-05-31 21:00:00,Coleraine,0.60
7246,2011-05-31 22:00:00,Coleraine,0.30


In [31]:
# Change Street type as string
df_CO_final['Street'] = df_CO_final['Street'].astype(str)

In [671]:
'''# Connect to postgres and create a database called 'project'
import psycopg2
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE pollution;')
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()
        '''

In [33]:
import psycopg2
# Create DB schema for CO readings and (DATE,STREET) as a composite primary key
createString = """
CREATE TABLE COpollution(
Date TIMESTAMP,
Street TEXT,
CarbonDioxide NUMERIC,
PRIMARY KEY (Date, Street)
);
"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [34]:
# Insert values from dataframe into database table
import pandas.io.sql as sqlio

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    for i,row in df_CO_final.iterrows():
        # on each row zeroth index has the date, 1st index has street and 2nd index has CarbonDioxide value
        insertString = "INSERT INTO COpollution VALUES('{}','{}','{}')".format(row[0],row[1],row[2])
        dbCursor.execute(insertString)
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [35]:
# Number of columns
col_names = df_CO_final.columns # make a list of all the column names in the dataframe 
totalcolumns = len(col_names)
totalcolumns

3

In [573]:
################################## Transforming SO2 DATASET ##################################

In [36]:
#inspect the dataframe for S02
df_SO2.head()

,Dublin City Council,Unnamed: 1,Coleraine Street,Unnamed: 3,Unnamed: 4,Unnamed: 5,Wood Quay (Winetavern St.),Unnamed: 7,Unnamed: 8,Unnamed: 9,,Unnamed: 11,Ballyfermot,Unnamed: 13
0,Site,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,Time,SO2 ug/m3,Daily Avg 24 hr.,Flag,Comment,SO2 ug/m3,Daily Avg 24 hr.,Flag,Comment,SO2 ug/m3,Daily Avg 24 hr.,Flag,Comment
2,01/01/2011,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2011,1.00,2.2,NaN,2,NaN,0.2,NaN,2,NaN,NaN,NaN,NaN,NaN
4,01/01/2011,2.00,2.0,NaN,2,NaN,0.0,NaN,2,NaN,NaN,NaN,NaN,NaN


In [37]:
df_SO2 = df_SO2[:3626] #keep top 3626 rows

In [38]:
# Drop first 2 rows 
df_SO2.drop(df_SO2.index[:2], inplace=True)
df_SO2.head()

,Dublin City Council,Unnamed: 1,Coleraine Street,Unnamed: 3,Unnamed: 4,Unnamed: 5,Wood Quay (Winetavern St.),Unnamed: 7,Unnamed: 8,Unnamed: 9,,Unnamed: 11,Ballyfermot,Unnamed: 13
2,01/01/2011,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2011,1.00,2.2,NaN,2,NaN,0.2,NaN,2,NaN,NaN,NaN,NaN,NaN
4,01/01/2011,2.00,2.0,NaN,2,NaN,0.0,NaN,2,NaN,NaN,NaN,NaN,NaN
5,01/01/2011,3.00,1.7,NaN,2,NaN,0.1,NaN,2,NaN,NaN,NaN,NaN,NaN
6,01/01/2011,4.00,1.7,NaN,2,NaN,0.0,NaN,2,NaN,NaN,NaN,NaN,NaN


In [39]:
# Based on column index, keep only date, time and SO2 of the 2 Streets
df_SO2_int = df_SO2.iloc[:,[0,1,6,2]] # same order with df_CO

In [40]:
print(df_SO2_int.columns)

Index(['Dublin City Council', 'Unnamed: 1',
       '                        Wood Quay (Winetavern St.)',
       '                  Coleraine Street '],
      dtype='object')


In [41]:
# Rename the remaining columns
col_rename_2 = {'Dublin City Council': 'Date', 'Unnamed: 1': 'Time', 
                '                        Wood Quay (Winetavern St.)': 'WinetavernSt', '                  Coleraine Street ': 'ColeraineSt'} 
df_SO2_renamed = df_SO2_int.rename(columns=col_rename_2)
df_SO2_renamed.head()

,Date,Time,WinetavernSt,ColeraineSt
2,01/01/2011,0.00,NaN,NaN
3,01/01/2011,1.00,0.2,2.2
4,01/01/2011,2.00,0.0,2.0
5,01/01/2011,3.00,0.1,1.7
6,01/01/2011,4.00,0.0,1.7


In [42]:
# Start index from 0 instead of 2
df_SO2_renamed.reset_index(drop=True, inplace=True)

In [43]:
# get dimensions of transformed dataset
print(df_SO2_renamed.shape)
display_n(df_SO2_renamed,5) # 3624 rows, same period with df_CO_renamed

(3624, 4)


,Date,Time,WinetavernSt,ColeraineSt
0,01/01/2011,0.00,NaN,NaN
1,01/01/2011,1.00,0.2,2.2
2,01/01/2011,2.00,0.0,2.0
3,01/01/2011,3.00,0.1,1.7
4,01/01/2011,4.00,0.0,1.7
...,...,...,...,...
3619,31/05/2011,19.00,0.1,0.0
3620,31/05/2011,20.00,0.0,0.0
3621,31/05/2011,21.00,0.0,0.0
3622,31/05/2011,22.00,0.0,0.0


In [44]:
# Find the special characters/non numeric values in the dataframe SO2
print(df_SO2_renamed.iloc[:,2].unique())
print(df_SO2_renamed.iloc[:,3].unique())

[nan '0.2' '0.0' '0.1' '-0.1' '-0.2' '-0.3' '0.3' '0.5' '0.4' '1.9' '1.8'
 '0.8' 'CAL' 'Data missing' '-0.4' '-0.5' '0.9' '2.9' '3.8' '4.4' '4.6'
 '1.7' '3.2' '3.0' '2.6' '0.6' '1.3' '0.7' '2.2' '4.8' '4.0' '5.3' '2.7'
 '3.3' '3.5' '6.2' '7.4' '7.9' '7.7' '7.8' '6.7' '5.6' '5.5' '6.5' '7.5'
 '8.8' '6.1' '4.1' '1.2' '1.5' '2.0' '6.4' '5.1' '3.4' '2.1' '1.1' '2.3'
 '1.6' '3.7' '5.0' '2.5' '1.0' '3.6' '1.4' '3.1' '4.7' '7.3' '8.4'
 'Data Missing' '2.8' '2.4' '4.3' '4.5' 'No data' 'Service' '3.9' '5.2'
 '6.8' '-0.6' '-0.7' '-0.8' '-0.9']
[nan '2.2' '2.0' '1.7' '1.5' '1.4' '1.0' '0.8' '0.7' '0.5' '0.4' '0.2'
 '0.3' '0.6' '0.9' '1.1' '1.6' '1.3' '1.2' '0.1' '0.0' '11.1' 'cal' '3.2'
 '4.0' '4.7' '3.3' '1.9' '3.4' '3.0' '4.3' 'no data' '2.5' '3.6' '3.7'
 '4.6' '5.4' '2.1' '2.7' '4.8' '4.5' '5.0' '5.6' '6.2' '6.0' '5.8' '5.5'
 '4.9' '4.1' '6.3' '7.7' '6.1' '5.3' '2.6' '2.3' '3.8' '2.9' '2.8' '2.4'
 '1.8' '3.1' '4.2' '6.8' '8.0' '11.3' '9.7' '9.4' '3.5' '-0.1' '-0.2'
 '5.7' '-0.3' '-0.4' '-0.5' 

In [45]:
# Find rows with special characters in WinetavernSt
print(df_SO2_renamed[df_SO2_renamed.WinetavernSt== 'CAL'])
print(df_SO2_renamed[df_SO2_renamed.WinetavernSt== 'Data missing'])
print(df_SO2_renamed[df_SO2_renamed.WinetavernSt== 'Data Missing'])
print(df_SO2_renamed[df_SO2_renamed.WinetavernSt== 'No data'])
print(df_SO2_renamed[df_SO2_renamed.WinetavernSt== 'Service'])

# Find rows with special characters in ColeraineSt
print(df_SO2_renamed[df_SO2_renamed.ColeraineSt== 'cal'])
print(df_SO2_renamed[df_SO2_renamed.ColeraineSt== 'no data'])
print(df_SO2_renamed[df_SO2_renamed.ColeraineSt== 'No data'])

            Date   Time WinetavernSt ColeraineSt
83    04/01/2011  11.00          CAL         0.5
84    04/01/2011  12.00          CAL         0.6
85    04/01/2011  13.00          CAL         0.6
86    04/01/2011  14.00          CAL         0.4
780   02/02/2011  12.00          CAL         0.5
781   02/02/2011  13.00          CAL         0.4
2271  05/04/2011  15.00          CAL         0.0
2272  05/04/2011  16.00          CAL        -0.2
2273  05/04/2011  17.00          CAL        -0.1
2942  03/05/2011  14.00          CAL         1.4
2943  03/05/2011  15.00          CAL         1.8
           Date   Time  WinetavernSt ColeraineSt
91   04/01/2011  19.00  Data missing         0.2
92   04/01/2011  20.00  Data missing         0.2
93   04/01/2011  21.00  Data missing         0.3
94   04/01/2011  22.00  Data missing         0.2
95   04/01/2011  23.00  Data missing         0.3
..          ...    ...           ...         ...
628  27/01/2011   4.00  Data missing         0.0
630  27/01/2011   6.

In [46]:
spec_char_2 = ['CAL','Data missing','Data Missing','Service','cal','no data','No data']
for i in spec_char_2:
    df_SO2_renamed = df_SO2_renamed.applymap(lambda x: np.NaN if (type(x) is str and i in x) else x)
    print(df_SO2_renamed[df_SO2_renamed.WinetavernSt == i])
    print(df_SO2_renamed[df_SO2_renamed.ColeraineSt == i])

Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: 

In [47]:
# Find the total rows with missing values from the transformed dataframe
df_SO2_renamed[df_SO2_renamed.isna().any(axis=1)]
# 1175 rows from 3624

,Date,Time,WinetavernSt,ColeraineSt
0,01/01/2011,0.00,NaN,NaN
83,04/01/2011,11.00,NaN,0.5
84,04/01/2011,12.00,NaN,0.6
85,04/01/2011,13.00,NaN,0.6
86,04/01/2011,14.00,NaN,0.4
...,...,...,...,...
3608,31/05/2011,8.00,-0.1,NaN
3610,31/05/2011,10.00,-0.4,NaN
3611,31/05/2011,11.00,0.0,NaN
3614,31/05/2011,14.00,-0.1,NaN


In [48]:
# find out how many values are missing
df_SO2_renamed.isna().sum().sum()
# total of 1,182 missing out of 14,496 observations

1182

In [49]:
# Drop Date column for groupby Time calculation of mean
df_SO2_trans = df_SO2_renamed.drop(df_SO2_renamed.columns[:1], axis = 1, inplace=False)
# Convert format of SO2 variables into numeric
df_SO2_trans[["WinetavernSt"]] = df_SO2_trans[["WinetavernSt"]].apply(pd.to_numeric)
df_SO2_trans[["ColeraineSt"]] = df_SO2_trans[["ColeraineSt"]].apply(pd.to_numeric)
df_SO2_trans[["Time"]] = df_SO2_trans[["Time"]].apply(pd.to_numeric)
# Assess if group means by Time have differences
print(df_SO2_trans.groupby('Time').aggregate('mean'))
# Replace NAs/missing values with group means
df_SO2_trans["WinetavernSt"] = df_SO2_trans.groupby("Time")["WinetavernSt"].transform(lambda x: x.fillna(x.mean()))
df_SO2_trans["ColeraineSt"] = df_SO2_trans.groupby("Time")["ColeraineSt"].transform(lambda x: x.fillna(x.mean()))

      WinetavernSt  ColeraineSt
Time                           
0.0       0.470270     0.365878
1.0       0.466667     0.330606
2.0       0.410280     0.230519
3.0       0.398182     0.238077
4.0       0.388073     0.251298
5.0       0.364545     0.449692
6.0       0.374074     0.581194
7.0       0.528696     0.647941
8.0       0.673504     0.612199
9.0       0.643697     0.618750
10.0      0.466116     0.584179
11.0      0.316102     0.600682
12.0      0.238938     0.544286
13.0      0.161404     0.658849
14.0      0.145455     0.575944
15.0      0.173394     0.761458
16.0      0.163478     0.796054
17.0      0.175000     0.681701
18.0      0.287023     0.482345
19.0      0.356589     0.321370
20.0      0.353390     0.537518
21.0      0.339130     0.539926
22.0      0.348718     0.454000
23.0      0.469298     0.404044


In [50]:
# find out how many values of the transformed dataset are missing
df_SO2_trans.isna().sum().sum()
# 0 missing values

0

In [51]:
print(df_SO2_trans.head())
print(df_SO2_trans.shape)

   Time  WinetavernSt  ColeraineSt
0   0.0       0.47027     0.365878
1   1.0       0.20000     2.200000
2   2.0       0.00000     2.000000
3   3.0       0.10000     1.700000
4   4.0       0.00000     1.700000
(3624, 3)


In [52]:
# Create a new variable called date_time which combines Date and Time from 2011/01/01 00:00:00 to 2011/05/31 23:00:00 
df_SO2_trans.loc[:,'date_time'] = pd.date_range('2011/01/01', periods = 3624, freq ='H')

In [53]:
# Drop Time variable
df_SO2_trans.drop(df_SO2_trans.columns[[0]], axis = 1, inplace=True)
display_n(df_SO2_trans,10)

,WinetavernSt,ColeraineSt,date_time
0,0.47027,0.365878,2011-01-01 00:00:00
1,0.20000,2.200000,2011-01-01 01:00:00
2,0.00000,2.000000,2011-01-01 02:00:00
3,0.10000,1.700000,2011-01-01 03:00:00
4,0.00000,1.700000,2011-01-01 04:00:00
...,...,...,...
3619,0.10000,0.000000,2011-05-31 19:00:00
3620,0.00000,0.000000,2011-05-31 20:00:00
3621,0.00000,0.000000,2011-05-31 21:00:00
3622,0.00000,0.000000,2011-05-31 22:00:00


In [54]:
# Intermediate dataframes for concatenating them
df_SO2_Wine = pd.concat([df_SO2_trans['date_time'], df_SO2_trans['WinetavernSt']], axis=1, keys=['date_time','WinetavernSt'])
df_SO2_Cole = pd.concat([df_SO2_trans['date_time'], df_SO2_trans['ColeraineSt']], axis=1, keys=['date_time','ColeraineSt'])
# Rename columns for values as SO2
df_SO2_Wine = df_SO2_Wine.rename(columns={"WinetavernSt": "SO2"})
df_SO2_Cole = df_SO2_Cole.rename(columns={"ColeraineSt": "SO2"})
# Create a variable as an ID for street
df_SO2_Wine.loc[:,'Street'] = 'Winetavern'
df_SO2_Cole.loc[:,'Street'] = 'Coleraine'
print(df_SO2_Wine.head())
print(df_SO2_Cole.head())

            date_time      SO2      Street
0 2011-01-01 00:00:00  0.47027  Winetavern
1 2011-01-01 01:00:00  0.20000  Winetavern
2 2011-01-01 02:00:00  0.00000  Winetavern
3 2011-01-01 03:00:00  0.10000  Winetavern
4 2011-01-01 04:00:00  0.00000  Winetavern
            date_time       SO2     Street
0 2011-01-01 00:00:00  0.365878  Coleraine
1 2011-01-01 01:00:00  2.200000  Coleraine
2 2011-01-01 02:00:00  2.000000  Coleraine
3 2011-01-01 03:00:00  1.700000  Coleraine
4 2011-01-01 04:00:00  1.700000  Coleraine


In [55]:
# Create a final concatenated dataset with an ID variable called "Street"
df_SO2_final = pd.concat([df_SO2_Wine, df_SO2_Cole], axis=0)
df_SO2_final = df_SO2_final[['date_time', 'Street','SO2']]
# Reset index in order to have 0:7247
df_SO2_final.reset_index(drop=True, inplace=True)
display_n(df_SO2_final,10)

,date_time,Street,SO2
0,2011-01-01 00:00:00,Winetavern,0.47027
1,2011-01-01 01:00:00,Winetavern,0.20000
2,2011-01-01 02:00:00,Winetavern,0.00000
3,2011-01-01 03:00:00,Winetavern,0.10000
4,2011-01-01 04:00:00,Winetavern,0.00000
...,...,...,...
7243,2011-05-31 19:00:00,Coleraine,0.00000
7244,2011-05-31 20:00:00,Coleraine,0.00000
7245,2011-05-31 21:00:00,Coleraine,0.00000
7246,2011-05-31 22:00:00,Coleraine,0.00000


In [56]:
# Change Street type as string
df_SO2_final['Street'] = df_SO2_final['Street'].astype(str)

In [57]:
# Create DB schema for SO2 readings and (DATE,STREET) as a composite primary key
createString = """
CREATE TABLE SO2pollution(
Date TIMESTAMP,
Street TEXT,
SulphurDioxide NUMERIC,
PRIMARY KEY (Date, Street)
);
"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [58]:
# Insert values from dataframe into database table
import pandas.io.sql as sqlio

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    for i,row in df_SO2_final.iterrows():
        # on each row zeroth index has the date, 1st index has street and 2nd index has SulphurDioxide value
        insertString = "INSERT INTO SO2pollution VALUES('{}','{}','{}')".format(row[0],row[1],row[2])
        dbCursor.execute(insertString)
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [59]:
# Number of columns, same with df_SO2_final
col_names = df_SO2_final.columns # make a list of all the column names in the dataframe 
totalcolumns = len(col_names)
totalcolumns

3

In [60]:
################################## Transforming NO2 DATASET ##################################

In [61]:
#inspect the dataframe for N02 
df_NO2.head()

,Site,Unnamed: 1,College Street,Unnamed: 3,Ballyfermot MPS,Unnamed: 5,Unnamed: 6,Unnamed: 7,Coleraine Street,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Wood Quay (Winetavern St.),Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,Date,Time,NO2 ug/m3,NO ug/m3,NOx ug/m3,NO2 Check,Flag,Comment,NO2 ug/m3,NO ug/m3,...,Flag,Comment,NO2 ug/m3,NO ug/m3,NOx ug/m3,NO2 Check,Flag,Comment,NaN,NaN
1,01/01/2011,01:00,NaN,NaN,NaN,0,NaN,NaN,23.7,5.3,...,2,NaN,48.9,37.5,86.5,48.9,2,NaN,NaN,NaN
2,01/01/2011,02:00,NaN,NaN,NaN,0,NaN,NaN,24.0,6.5,...,2,NaN,44.1,17.8,61.9,44.1,2,NaN,NaN,NaN
3,01/01/2011,03:00,NaN,NaN,NaN,0,NaN,NaN,26.8,12.8,...,2,NaN,35.0,5.9,41.0,35.0,2,NaN,NaN,NaN
4,01/01/2011,04:00,NaN,NaN,NaN,0,NaN,NaN,27.0,12.8,...,2,NaN,38.3,10.7,49.0,38.3,2,NaN,NaN,NaN


In [62]:
#keep top 3624 rows
df_NO2 = df_NO2[:3624] 
# Drop first 1 row 
df_NO2.drop(df_NO2.index[:1], inplace=True)
# Based on column index, keep only date, time and NO2 of the 2 Streets
df_NO2_int = df_NO2.iloc[:,[0,1,14,8]] # same order with the other 2 datasets
print(df_NO2_int.columns)

Index(['Site', 'Unnamed: 1',
       '                              Wood Quay (Winetavern St.)',
       '                                           Coleraine Street '],
      dtype='object')


In [63]:
# Rename the remaining columns
col_rename_3 = {'Site': 'Date', 'Unnamed: 1': 'Time', 
                '                              Wood Quay (Winetavern St.)': 'WinetavernSt', '                                           Coleraine Street ': 'ColeraineSt'} 
df_NO2_renamed = df_NO2_int.rename(columns=col_rename_3)
df_NO2_renamed.head()

,Date,Time,WinetavernSt,ColeraineSt
1,01/01/2011,01:00,48.9,23.7
2,01/01/2011,02:00,44.1,24.0
3,01/01/2011,03:00,35.0,26.8
4,01/01/2011,04:00,38.3,27.0
5,01/01/2011,05:00,35.5,19.3


In [64]:
# Start index from 0 instead of 1
df_NO2_renamed.reset_index(drop=True, inplace=True)

In [65]:
# get dimensions of transformed dataset
print(df_NO2_renamed.shape)
display_n(df_NO2_renamed,5) # 3623 rows, same period-(1 hour less-first entry) with the other 2

(3623, 4)


,Date,Time,WinetavernSt,ColeraineSt
0,01/01/2011,01:00,48.9,23.7
1,01/01/2011,02:00,44.1,24.0
2,01/01/2011,03:00,35.0,26.8
3,01/01/2011,04:00,38.3,27.0
4,01/01/2011,05:00,35.5,19.3
...,...,...,...,...
3618,31/05/2011,19:00,28.9,5.7
3619,31/05/2011,20:00,25.9,8.2
3620,31/05/2011,21:00,23.4,14.1
3621,31/05/2011,22:00,28.3,11.3


In [66]:
# Find the special characters/non numeric values in the dataframe NO2
print(df_NO2_renamed.iloc[:,2].unique())
print(df_NO2_renamed.iloc[:,3].unique())

['48.9' '44.1' '35.0' '38.3' '35.5' '33.7' '38.4' '30.2' '32.4' '27.1'
 '29.8' '25.3' '25.2' '22.0' '24.9' '32.7' '37.2' '30.3' '36.5' '34.6'
 '31.3' '26.1' '21.9' '14.1' '13.7' '12.2' '7.5' '9.4' '20.9' '32.9'
 '23.6' '28.3' '33.8' '32.1' '29.2' '44.3' '48.2' '48.1' '47.0' '49.5'
 '52.1' '53.7' '40.2' '33.5' '39.5' '30.9' '25.4' '24.7' '26.0' '43.8'
 '43.2' '49.2' '51.2' '44.7' '29.6' '52.3' '73.9' '82.5' '78.7' '56.5'
 '27.5' '19.6' '15.2' '11.0' '7.8' '7.2' '6.8' '6.2' '7.7' '17.7' '40.5'
 '35.9' 'CAL' '37.6' '37.5' '43.0' '45.0' '35.3' '23.1' '20.4' '15.6'
 '13.3' '11.4' '16.0' '11.8' '7.0' '6.6' '10.0' '49.7' '46.5' '34.0'
 '42.3' '50.2' '59.9' '68.6' '69.3' '61.5' '62.5' '57.8' '58.8' '59.5'
 '59.7' '49.8' '46.9' '37.3' '35.1' '68.3' '74.9' '74.0' '71.3' '66.7'
 '64.0' '63.7' '69.6' '78.1' '92.6' '93.3' '95.8' '94.3' '108.9' '110.5'
 '109.1' '110.7' '88.9' '83.6' '87.8' '84.1' '50.5' '69.7' '76.1' '85.0'
 '75.5' '76.2' '69.9' '72.1' '70.2' '73.8' '83.9' '82.0' '63.4' '59.1'
 '55.

In [67]:
# Find rows with special characters in WinetavernSt
print(df_NO2_renamed[df_NO2_renamed.WinetavernSt== 'CAL'])
print(df_NO2_renamed[df_NO2_renamed.WinetavernSt== 'No data'])
print(df_NO2_renamed[df_NO2_renamed.WinetavernSt== 'Service'])
print(df_NO2_renamed[df_NO2_renamed.WinetavernSt== 'Cal'])
# Find rows with special characters in ColeraineSt
print(df_NO2_renamed[df_NO2_renamed.ColeraineSt== 'cal'])
print(df_NO2_renamed[df_NO2_renamed.ColeraineSt== 'no data'])

            Date   Time WinetavernSt ColeraineSt
82    04/01/2011  11:00          CAL        31.0
83    04/01/2011  12:00          CAL        33.5
84    04/01/2011  13:00          CAL        41.8
779   02/02/2011  12:00          CAL        19.8
780   02/02/2011  13:00          CAL        25.3
1667  11/03/2011  12:00          CAL        55.2
1668  11/03/2011  13:00          CAL        54.1
1669  11/03/2011  14:00          CAL        44.1
2270  05/04/2011  15:00          CAL        17.4
2271  05/04/2011  16:00          CAL        16.2
2941  03/05/2011  14:00          CAL        36.0
2942  03/05/2011  15:00          CAL        37.0
            Date   Time WinetavernSt ColeraineSt
1642  10/03/2011  11:00      No data        20.0
2041  27/03/2011  02:00      No data     no data
            Date   Time WinetavernSt ColeraineSt
1761  15/03/2011  10:00      Service        41.3
1762  15/03/2011  11:00      Service        40.0
1763  15/03/2011  12:00      Service        38.9
1764  15/03/2011  13

In [68]:
spec_char_3 = ['CAL','Cal','Service','cal','no data','No data']
for i in spec_char_3:
    df_NO2_renamed = df_NO2_renamed.applymap(lambda x: np.NaN if (type(x) is str and i in x) else x)
    print(df_NO2_renamed[df_NO2_renamed.WinetavernSt == i])
    print(df_NO2_renamed[df_NO2_renamed.ColeraineSt == i])

Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []
Empty DataFrame
Columns: [Date, Time, WinetavernSt, ColeraineSt]
Index: []


In [69]:
# Find the total rows with missing values from the transformed dataframe
df_NO2_renamed[df_NO2_renamed.isna().any(axis=1)]
# 38 rows from 3623

,Date,Time,WinetavernSt,ColeraineSt
82,04/01/2011,11:00,NaN,31.0
83,04/01/2011,12:00,NaN,33.5
84,04/01/2011,13:00,NaN,41.8
109,05/01/2011,14:00,34.0,NaN
426,18/01/2011,19:00,93.4,NaN
427,18/01/2011,20:00,86.6,NaN
754,01/02/2011,11:00,61.3,NaN
755,01/02/2011,12:00,48.5,NaN
779,02/02/2011,12:00,NaN,19.8
780,02/02/2011,13:00,NaN,25.3


In [70]:
# find out how many values are missing
df_NO2_renamed.isna().sum().sum()
# total of 38 missing out of 14,492 observations

38

In [71]:
# Drop Date column for groupby Time calculation of mean
df_NO2_trans = df_NO2_renamed.drop(df_NO2_renamed.columns[:1], axis = 1, inplace=False)
# Convert format of NO2 variables into numeric
df_NO2_trans[["WinetavernSt"]] = df_NO2_trans[["WinetavernSt"]].apply(pd.to_numeric)
df_NO2_trans[["ColeraineSt"]] = df_NO2_trans[["ColeraineSt"]].apply(pd.to_numeric)
# Assess if group means by Time have differences
print(df_NO2_trans.groupby('Time').aggregate('mean'))
# Replace NAs/missing values with group means
df_NO2_trans["WinetavernSt"] = df_NO2_trans.groupby("Time")["WinetavernSt"].transform(lambda x: x.fillna(x.mean()))
df_NO2_trans["ColeraineSt"] = df_NO2_trans.groupby("Time")["ColeraineSt"].transform(lambda x: x.fillna(x.mean()))

       WinetavernSt  ColeraineSt
Time                            
01:00     36.703311    27.929139
02:00     34.096000    25.321333
03:00     30.792053    25.038411
04:00     29.652318    23.854305
05:00     27.796026    27.192053
06:00     28.509934    35.392053
07:00     38.802649    44.609272
08:00     51.984106    46.727152
09:00     56.587417    42.496689
10:00     53.907333    36.657333
11:00     50.364865    31.414865
12:00     44.859184    29.820408
13:00     43.934694    29.903356
14:00     43.043919    30.011409
15:00     44.390541    30.821333
16:00     46.922819    34.104000
17:00     52.425166    38.109934
18:00     54.728477    40.447682
19:00     53.670199    40.521333
20:00     50.691391    41.743333
21:00     49.586093    41.697351
22:00     47.753642    39.775497
23:00     45.980795    35.554967
24:00     41.388667    31.690000


In [72]:
# find out how many values of the transformed dataset are missing
df_NO2_trans.isna().sum().sum()
# 0 missing values

0

In [73]:
print(df_NO2_trans.head())
print(df_NO2_trans.shape)

    Time  WinetavernSt  ColeraineSt
0  01:00          48.9         23.7
1  02:00          44.1         24.0
2  03:00          35.0         26.8
3  04:00          38.3         27.0
4  05:00          35.5         19.3
(3623, 3)


In [74]:
# Create a new variable called date_time which combines Date and Time from 2011/01/01 01:00:00 to 2011/05/31 23:00:00 
df_int = pd.date_range('2011/01/01', periods = 3624, freq ='H')
mask = (df_int > '2011-01-01 00:00:00') & (df_int <= '2011-05-31 23:00:00')
df_NO2_trans.loc[:,'date_time'] = df_int[mask]

In [75]:
# Drop Time variable
df_NO2_trans.drop(df_NO2_trans.columns[[0]], axis = 1, inplace=True)
display_n(df_NO2_trans,10)

,WinetavernSt,ColeraineSt,date_time
0,48.9,23.7,2011-01-01 01:00:00
1,44.1,24.0,2011-01-01 02:00:00
2,35.0,26.8,2011-01-01 03:00:00
3,38.3,27.0,2011-01-01 04:00:00
4,35.5,19.3,2011-01-01 05:00:00
...,...,...,...
3618,28.9,5.7,2011-05-31 19:00:00
3619,25.9,8.2,2011-05-31 20:00:00
3620,23.4,14.1,2011-05-31 21:00:00
3621,28.3,11.3,2011-05-31 22:00:00


In [76]:
# Intermediate dataframes for concatenating them
df_NO2_Wine = pd.concat([df_NO2_trans['date_time'], df_NO2_trans['WinetavernSt']], axis=1, keys=['date_time','WinetavernSt'])
df_NO2_Cole = pd.concat([df_NO2_trans['date_time'], df_NO2_trans['ColeraineSt']], axis=1, keys=['date_time','ColeraineSt'])
# Rename columns for values as NO2
df_NO2_Wine = df_NO2_Wine.rename(columns={"WinetavernSt": "NO2"})
df_NO2_Cole = df_NO2_Cole.rename(columns={"ColeraineSt": "NO2"})
# Create a variable as an ID for street
df_NO2_Wine.loc[:,'Street'] = 'Winetavern'
df_NO2_Cole.loc[:,'Street'] = 'Coleraine'
print(df_NO2_Wine.head())
print(df_NO2_Cole.head())


            date_time   NO2      Street
0 2011-01-01 01:00:00  48.9  Winetavern
1 2011-01-01 02:00:00  44.1  Winetavern
2 2011-01-01 03:00:00  35.0  Winetavern
3 2011-01-01 04:00:00  38.3  Winetavern
4 2011-01-01 05:00:00  35.5  Winetavern
            date_time   NO2     Street
0 2011-01-01 01:00:00  23.7  Coleraine
1 2011-01-01 02:00:00  24.0  Coleraine
2 2011-01-01 03:00:00  26.8  Coleraine
3 2011-01-01 04:00:00  27.0  Coleraine
4 2011-01-01 05:00:00  19.3  Coleraine


In [77]:
# Create a final concatenated dataset with an ID variable called "Street"
df_NO2_final = pd.concat([df_NO2_Wine, df_NO2_Cole], axis=0)
df_NO2_final = df_NO2_final[['date_time', 'Street','NO2']]
# Reset index in order to have 0:7245
df_NO2_final.reset_index(drop=True, inplace=True)
display_n(df_NO2_final,10)

,date_time,Street,NO2
0,2011-01-01 01:00:00,Winetavern,48.9
1,2011-01-01 02:00:00,Winetavern,44.1
2,2011-01-01 03:00:00,Winetavern,35.0
3,2011-01-01 04:00:00,Winetavern,38.3
4,2011-01-01 05:00:00,Winetavern,35.5
...,...,...,...
7241,2011-05-31 19:00:00,Coleraine,5.7
7242,2011-05-31 20:00:00,Coleraine,8.2
7243,2011-05-31 21:00:00,Coleraine,14.1
7244,2011-05-31 22:00:00,Coleraine,11.3


In [78]:
# Change Street type as string
df_NO2_final['Street'] = df_NO2_final['Street'].astype(str)

In [79]:
# Create DB schema for NO2 readings and (DATE,STREET) as a composite primary key
createString = """
CREATE TABLE NO2pollution(
Date TIMESTAMP,
Street TEXT,
NitrogenDioxide NUMERIC,
PRIMARY KEY (Date, Street)
);
"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [80]:
# Insert values from dataframe into database table
import pandas.io.sql as sqlio

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    for i,row in df_NO2_final.iterrows():
        # on each row zeroth index has the date, 1st index has street and 2nd index has NitrogenDioxide value
        insertString = "INSERT INTO NO2pollution VALUES('{}','{}','{}')".format(row[0],row[1],row[2])
        dbCursor.execute(insertString)
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [81]:
# Number of columns, same with df_NO2_final
col_names = df_NO2_final.columns # make a list of all the column names in the dataframe 
totalcolumns = len(col_names)
totalcolumns

3

In [843]:
################################## ANALYSIS DATASET AIRPOLLUTION ##################################

In [82]:
# Create table airpollution with an inner join on the composite primary key (DATE,STREET)
createString = """
CREATE TABLE airpollution as (
    select 
        a.Date ,
        a.Street ,
        a.CarbonDioxide,
        b.SulphurDioxide ,
        c.NitrogenDioxide
    from copollution as a
    left join so2pollution as b
    on a.Date=b.Date and a.Street=b.Street
    left join no2pollution as c
    on b.Date=c.Date and b.Street=c.Street
);
"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [ ]:
#Start of Ian's Analysis: Temperature vs Energy Expenditure

In [276]:
analysisQuery = """
SELECT time, apparentTemperature as app_temp, cloudCover as cloud, humidity,
        precipIntensity as precip, pressure, temperature as act_temp, uvIndex,
        visibility, windSpeed, apparentTemperature - temperature as temp_dif,
        hdd
FROM weather
LEFT JOIN hourlyhdd ON weather.time = hourlyhdd.date_time
"""

In [277]:
analysis_df = enter_query("dap_project", analysisQuery)

In [278]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import chart_studio.plotly as py

In [279]:
analysis_df.head()

,time,app_temp,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed,temp_dif,hdd
0,2011-01-01 00:00:00,4.21,0.81,0.80,0.0,1027.8,6.38,0,8.998,2.93,-2.17,0.359363
1,2011-01-01 01:00:00,3.36,0.78,0.89,0.0,1027.8,5.67,0,8.005,2.93,-2.31,0.364893
2,2011-01-01 02:00:00,3.08,0.78,0.84,0.0,1027.5,5.65,0,8.005,3.29,-2.57,0.367193
3,2011-01-01 03:00:00,3.06,0.78,0.89,0.0,1027.4,5.63,0,8.005,3.29,-2.57,0.365805
4,2011-01-01 04:00:00,3.06,0.78,0.90,0.0,1027.4,5.63,0,8.005,3.29,-2.57,0.357785


In [280]:
#as this was a left join, there are values for temp where there are
#no values for hdd. I filter these instances out here
temp_hdd = analysis_df.iloc[:, [0,6,11]][analysis_df.hdd.notnull()]

In [281]:
temp_hdd.set_index("time", inplace = True)

In [282]:
temp_hdd.dtypes

act_temp    float64
hdd         float64
dtype: object

In [283]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = temp_hdd.index, y = temp_hdd["act_temp"], mode = "lines", name ="temperature"))
fig.add_trace(go.Scatter(x = temp_hdd.index, y = temp_hdd["hdd"], mode = "lines", name = "energy expenditure"))
fig.show()

In [284]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
print(scaler.fit(temp_hdd))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [285]:
scaled = scaler.transform(temp_hdd)

In [286]:
temp_hdd["scaled_temp"] = scaled[:, 0]
temp_hdd["scaled_hdd"] = scaled[:, 1]

In [287]:
temp_hdd["temp_sma12"] = temp_hdd.iloc[:, 2].rolling(window = 12).mean()
temp_hdd["hdd_sma12"] = temp_hdd.iloc[:, 3].rolling(window = 12).mean()

In [420]:
fig = go.Figure()

original_hdd_trace = go.Scatter(
x = temp_hdd.index,
y = temp_hdd["hdd"],
mode = "lines",
name = "Energy Expenditure")

original_temp_trace = go.Scatter(
x = temp_hdd.index,
y = temp_hdd["act_temp"],
mode = "lines",
name = "Temperature")



hdd_trace = go.Scatter(
    x = temp_hdd.index,
    y = temp_hdd["hdd_sma12"],
    mode = "lines",
    name = "Energy Expenditure")

temp_trace = go.Scatter(
    x = temp_hdd.index,
    y = temp_hdd["temp_sma12"],
    mode = "lines",
    name = "Temperature")

inverse_temp_trace = go.Scatter(
    x = temp_hdd.index,
    y = temp_hdd["temp_sma12"].apply(lambda x: 1-x),
    mode = "lines",
    name = "Inverse Temperature")

data1 = [original_hdd_trace, original_temp_trace]
data2 = [hdd_trace, temp_trace]
data3 = [hdd_trace, inverse_temp_trace]

layout = go.Layout(
xaxis = dict(
autorange = True,
showgrid = False,
zeroline = True,
showline = True,
linecolor = "#000000"
),
yaxis = dict(
title = "Scaled Value",
autorange = True,
showgrid = False,
zeroline = True,
showline = True,
linecolor = "#000000"
),
yaxis2 = dict(
title = "other axis",
autorange = True,
showgrid = False,
zeroline = True,
showline = True,
linecolor = "#000000"
),
paper_bgcolor = "rgba(0,0,0,0)",
plot_bgcolor = "rgba(0,0,0,0)"
)


In [409]:
layout2 = layout
layout2["yaxis"]["title"]["text"] = "Temperature"

In [421]:
fig = go.Figure(data = data1, layout = layout)
fig.show()

In [289]:
fig = go.Figure(data = data1, layout = layout)
fig.show()

In [290]:
fig = go.Figure(data = data2, layout = layout)
fig.show()

In [291]:
#gets all of the analysis_df but stores it as a separate dataframe
act_app_df = analysis_df.iloc[:, :-1].copy()

In [292]:
analysis_df.head()

,time,app_temp,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed,temp_dif,hdd
0,2011-01-01 00:00:00,4.21,0.81,0.80,0.0,1027.8,6.38,0,8.998,2.93,-2.17,0.359363
1,2011-01-01 01:00:00,3.36,0.78,0.89,0.0,1027.8,5.67,0,8.005,2.93,-2.31,0.364893
2,2011-01-01 02:00:00,3.08,0.78,0.84,0.0,1027.5,5.65,0,8.005,3.29,-2.57,0.367193
3,2011-01-01 03:00:00,3.06,0.78,0.89,0.0,1027.4,5.63,0,8.005,3.29,-2.57,0.365805
4,2011-01-01 04:00:00,3.06,0.78,0.90,0.0,1027.4,5.63,0,8.005,3.29,-2.57,0.357785


In [293]:
act_app_df.set_index("time", inplace = True)

In [295]:
act_app_df.head()

,app_temp,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed,temp_dif
time,,,,,,,,,,
2011-01-01 00:00:00,4.21,0.81,0.80,0.0,1027.8,6.38,0,8.998,2.93,-2.17
2011-01-01 01:00:00,3.36,0.78,0.89,0.0,1027.8,5.67,0,8.005,2.93,-2.31
2011-01-01 02:00:00,3.08,0.78,0.84,0.0,1027.5,5.65,0,8.005,3.29,-2.57
2011-01-01 03:00:00,3.06,0.78,0.89,0.0,1027.4,5.63,0,8.005,3.29,-2.57
2011-01-01 04:00:00,3.06,0.78,0.90,0.0,1027.4,5.63,0,8.005,3.29,-2.57


In [296]:
#need to scale data
print(scaler.fit(act_app_df))
scaled_data = scaler.transform(act_app_df)

MinMaxScaler(copy=True, feature_range=(0, 1))


In [297]:
scaled_df = pd.DataFrame(scaled_data, columns = act_app_df.columns)

In [298]:
scaled_df.head()

,app_temp,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed,temp_dif
0,0.424166,0.81,0.701493,0.0,0.743557,0.441271,0.0,0.556396,0.160022,0.669711
1,0.393689,0.78,0.835821,0.0,0.743557,0.412711,0.0,0.494310,0.160022,0.648402
2,0.383650,0.78,0.761194,0.0,0.739691,0.411907,0.0,0.494310,0.179683,0.608828
3,0.382933,0.78,0.835821,0.0,0.738402,0.411102,0.0,0.494310,0.179683,0.608828
4,0.382933,0.78,0.850746,0.0,0.738402,0.411102,0.0,0.494310,0.179683,0.608828


In [299]:
scaled_dif_df = scaled_df.diff()

In [300]:
scaled_dif_df.head()

,app_temp,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed,temp_dif
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.030477,-0.03,0.134328,0.0,0.000000,-0.028560,0.0,-0.062086,0.000000,-0.021309
2,-0.010039,0.00,-0.074627,0.0,-0.003866,-0.000805,0.0,0.000000,0.019661,-0.039574
3,-0.000717,0.00,0.074627,0.0,-0.001289,-0.000805,0.0,0.000000,0.000000,0.000000
4,0.000000,0.00,0.014925,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [301]:
#Removed the first row, lost due to differencing
#Also removed apparent temperature, kept temperature to see if the
#actual temperature is a predictor for the difference between temp_dif
#would be redundant to keep both.
scaled_dif_df = scaled_dif_df.iloc[1:,1:]

In [302]:
scaled_dif_df.head()

,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed,temp_dif
1,-0.03,0.134328,0.0,0.000000,-0.028560,0.0,-0.062086,0.000000,-0.021309
2,0.00,-0.074627,0.0,-0.003866,-0.000805,0.0,0.000000,0.019661,-0.039574
3,0.00,0.074627,0.0,-0.001289,-0.000805,0.0,0.000000,0.000000,0.000000
4,0.00,0.014925,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,0.00,-0.074627,0.0,-0.003866,0.029364,0.0,0.062086,0.033861,-0.033486


In [303]:
dependent = scaled_dif_df.iloc[:, -1]
independent  = scaled_dif_df.iloc[:, :-1]

In [304]:
independent.head()

,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed
1,-0.03,0.134328,0.0,0.000000,-0.028560,0.0,-0.062086,0.000000
2,0.00,-0.074627,0.0,-0.003866,-0.000805,0.0,0.000000,0.019661
3,0.00,0.074627,0.0,-0.001289,-0.000805,0.0,0.000000,0.000000
4,0.00,0.014925,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5,0.00,-0.074627,0.0,-0.003866,0.029364,0.0,0.062086,0.033861


In [305]:
from sklearn.model_selection import train_test_split
ind_train, ind_test, dep_train, dep_test = train_test_split(independent,
                                                            dependent,
                                                            test_size = 0.2,
                                                           random_state = 0)


In [306]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [307]:
dep_train.head()

5592    0.668189
1706   -0.063927
2550    0.000000
296     0.000000
4171   -0.048706
Name: temp_dif, dtype: float64

In [308]:
ind_train.head()

,cloud,humidity,precip,pressure,act_temp,uvindex,visibility,windspeed
5592,0.61,0.164179,0.052597,0.217784,0.207562,0.0,0.0,0.192245
1706,0.00,0.074627,0.000000,0.001289,-0.032985,0.0,0.0,0.024577
2550,0.10,0.014925,0.000000,0.005155,-0.012068,0.0,0.0,-0.003277
296,-0.12,0.059701,0.004263,0.007732,-0.032985,0.0,0.0,0.065538
4171,0.36,-0.283582,0.033251,0.246134,0.163717,0.0,0.0,0.302021


In [309]:
regressor.fit(ind_train,dep_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [310]:
y_pred = regressor.predict(ind_test)

In [312]:
import statsmodels.api as sm

In [127]:
regressor_OLS = sm.OLS(endog = dep_train, exog = ind_train).fit()

In [128]:
#visibility no value
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               temp_dif   R-squared (uncentered):                   0.161
Model:                            OLS   Adj. R-squared (uncentered):              0.160
Method:                 Least Squares   F-statistic:                              114.4
Date:                Fri, 10 Apr 2020   Prob (F-statistic):                   1.91e-175
Time:                        15:44:10   Log-Likelihood:                          4095.8
No. Observations:                4759   AIC:                                     -8176.
Df Residuals:                    4751   BIC:                                     -8124.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cloud         -0.0112      0.015     -0.756      0.450      -0.040       0.018
humidity      -0.0700      0.033     -2.094      0.036      -0.136      -0.004
precip         0.0799      0.035      2.253      0.024       0.010       0.149
pressure      -0.3350      0.131     -2.557      0.011      -0.592      -0.078
act_temp       1.0756      0.063     17.050      0.000       0.952       1.199
uvindex        0.0562      0.025      2.246      0.025       0.007       0.105
visibility     0.0272      0.027      1.011      0.312      -0.026       0.080
windspeed     -0.6921      0.034    -20.550      0.000      -0.758      -0.626
==============================================================================
Omnibus:                     1150.303   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49451.594
Skew:                           0.351   Prob(JB):                         0.00
Kurtosis:                      18.776   Cond. No.                         9.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [129]:
ind2 = ind_train.iloc[:, [0,1,2,3,4,5,7]]

In [132]:
regressor_OLS2 = sm.OLS(endog = dep_train, exog = ind2).fit()
regressor_OLS2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               temp_dif   R-squared (uncentered):                   0.161
Model:                            OLS   Adj. R-squared (uncentered):              0.160
Method:                 Least Squares   F-statistic:                              130.6
Date:                Fri, 10 Apr 2020   Prob (F-statistic):                   2.68e-176
Time:                        15:44:45   Log-Likelihood:                          4095.3
No. Observations:                4759   AIC:                                     -8177.
Df Residuals:                    4752   BIC:                                     -8131.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cloud         -0.0114      0.015     -0.771      0.441      -0.040       0.018
humidity      -0.0719      0.033     -2.153      0.031      -0.137      -0.006
precip         0.0779      0.035      2.200      0.028       0.008       0.147
pressure      -0.3346      0.131     -2.554      0.011      -0.591      -0.078
act_temp       1.0780      0.063     17.100      0.000       0.954       1.202
uvindex        0.0561      0.025      2.241      0.025       0.007       0.105
windspeed     -0.6925      0.034    -20.561      0.000      -0.759      -0.626
==============================================================================
Omnibus:                     1149.244   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49485.262
Skew:                           0.349   Prob(JB):                         0.00
Kurtosis:                      18.782   Cond. No.                         9.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [133]:
#It seems that although each variable is quite significantly associated
#with the dependent varible, the R squared value of .16 suggests that
#these variables only explain a small proportion of the variance in
#the difference between actual and apparent temperature.
ind3 = ind_train.iloc[:, [1,2,3,4,5,7]]
regressor_OLS3 = sm.OLS(endog = dep_train, exog = ind3).fit()
regressor_OLS3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               temp_dif   R-squared (uncentered):                   0.161
Model:                            OLS   Adj. R-squared (uncentered):              0.160
Method:                 Least Squares   F-statistic:                              152.2
Date:                Fri, 10 Apr 2020   Prob (F-statistic):                   2.80e-177
Time:                        15:44:46   Log-Likelihood:                          4095.0
No. Observations:                4759   AIC:                                     -8178.
Df Residuals:                    4753   BIC:                                     -8139.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
humidity      -0.0743      0.033     -2.237      0.025      -0.140      -0.009
precip         0.0768      0.035      2.171      0.030       0.007       0.146
pressure      -0.3310      0.131     -2.528      0.012      -0.588      -0.074
act_temp       1.0728      0.063     17.115      0.000       0.950       1.196
uvindex        0.0572      0.025      2.289      0.022       0.008       0.106
windspeed     -0.6933      0.034    -20.599      0.000      -0.759      -0.627
==============================================================================
Omnibus:                     1149.375   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49468.772
Skew:                           0.349   Prob(JB):                         0.00
Kurtosis:                      18.779   Cond. No.                         6.56
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
from sklearn.metrics import mean_squared_error
from math import sqrt
ind3_test = ind_test.iloc[:, [1,2,3,4,5,7]]
regressor.fit(ind3, dep_train)
y_preds = regressor.predict(ind3_test)
rms = sqrt(mean_squared_error(dep_test, y_preds))

In [138]:
rms

0.10605170740877044

In [315]:
just_temps = analysis_df.iloc[:, [0, 6]].copy()

In [318]:
just_temps.dtypes

time        datetime64[ns]
act_temp           float64
dtype: object

In [321]:
just_temps["year"] = just_temps.time.apply(lambda x: x.strftime("%Y"))
just_temps["rest_date"] = just_temps.time.apply(lambda x: x.strftime("%m-%d %H:%M:%S"))

In [323]:
just_temps = just_temps.iloc[:, 1:]

In [335]:
just_temps.dtypes

act_temp    float64
year         object
dtype: object

In [345]:
temps_2011 = just_temps[just_temps["year"] =="2011"]
temps_2020 = just_temps[just_temps["year"] == "2020"]

In [387]:
#earlier we saw that temperature was too hard to distinguish,
#as data was too dense. Therefore smoothed it again, with period 12
temps_2011["sma_12"] = temps_2011.iloc[:, 0].rolling(window = 12).mean()
temps_2020["sma_12"] = temps_2020.iloc[:, 0].rolling(window = 12).mean()

In [388]:
#Will turn the first 12 values into nan
temps_2011.head(14)

,act_temp,year,sma_12,sma_72
rest_date,,,,
01-01 00:00:00,6.38,2011,NaN,NaN
01-01 01:00:00,5.67,2011,NaN,NaN
01-01 02:00:00,5.65,2011,NaN,NaN
01-01 03:00:00,5.63,2011,NaN,NaN
01-01 04:00:00,5.63,2011,NaN,NaN
01-01 05:00:00,6.36,2011,NaN,NaN
01-01 06:00:00,6.38,2011,NaN,NaN
01-01 07:00:00,6.43,2011,NaN,NaN
01-01 08:00:00,5.67,2011,NaN,NaN


In [389]:
temps_2020.head(14)

,act_temp,year,sma_12,sma_72
rest_date,,,,
01-24 16:00:00,7.20,2020,NaN,NaN
02-03 08:00:00,6.38,2020,NaN,NaN
01-26 05:00:00,10.00,2020,NaN,NaN
03-02 17:00:00,6.45,2020,NaN,NaN
02-10 03:00:00,5.03,2020,NaN,NaN
01-30 09:00:00,9.99,2020,NaN,NaN
04-02 14:00:00,12.88,2020,NaN,NaN
02-20 15:00:00,6.80,2020,NaN,NaN
02-02 20:00:00,9.65,2020,NaN,NaN


In [390]:
temps_2011.index[[360, 1080, 1800, 2520, 3240]]

Index(['01-16 00:00:00', '02-15 00:00:00', '03-17 00:00:00', '04-16 01:00:00',
       '05-16 01:00:00'],
      dtype='object', name='rest_date')

In [393]:
fig = go.Figure()

trace_2011 = go.Scatter(
    x = temps_2011.index,
    y = temps_2011["sma_12"],
    mode = "lines",
    name = "2011 Temperatures")

trace_2020 = go.Scatter(
    x = temps_2011.index,
    y = temps_2020["sma_12"],
    mode = "lines",
    name = "2020 Temperatures")


data = [trace_2011,trace_2020]

layout = go.Layout(
xaxis = dict(
autorange = True,
showgrid = False,
zeroline = True,
showline = True,
linecolor = "#000000",
tickmode = "array",
tickvals = temps_2011.index[[360, 1080, 1800, 2520, 3240]],
ticktext = ["January" , "February", "March", "April", "May"]
),
yaxis = dict(
title = "Temperature",
autorange = True,
showgrid = False,
zeroline = True,
showline = True,
linecolor = "#000000"
),
paper_bgcolor = "rgba(0,0,0,0)",
plot_bgcolor = "rgba(0,0,0,0)"
)

In [394]:
fig = go.Figure(data = data, layout = layout)
fig.show()